In [1]:
import utils
from torch.utils.data import DataLoader, RandomSampler
from torch.autograd import Variable
from dataset import *
from Enc_Dec import *
from model1 import *
from tokenizer_ import *
import random
from torch.autograd import Variable
from torch.optim import SGD
from attention_params import *
import argparse
import logging
from utils import *
import time
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter

import glob


In [2]:
#load data
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

max_length=10
min_freq=5
vocabsize=4000
save_location='./reading-by-translating/'
#train tokenizer
tokenizer = get_tokenizer(pairs, max_length, min_freq, vocabsize, save_location)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4346
eng 2804
['j ai les chocottes .', 'i m chicken .']
training tokenizers
{'##ese': 971, 'the': 97, 'aider': 856, 'vraiment': 353, 'chret': 3699, 'timide': 1595, 'ide': 1320, '##ff': 234, 'disponible': 3089, 'frightened': 1672, 'humeur': 2142, 'without': 2560, '##jourd': 806, 'act': 683, '##uges': 3629, 'bleeding': 3889, 'cuis': 1283, 'venir': 2149, 'constantly': 3876, '##riend': 318, '##ants': 535, 'gre': 1960, 'amies': 1891, 'idiot': 669, '##ust': 864, 'closet': 3842, '##satisf': 3240, 'late': 712, '##ven': 788, 'completely': 1531, 'genie': 3772, 'powerful': 1944, 'worried': 783, '##ossible': 2113, '##ur': 137, '[S]': 0, 'surv': 2115, 'socc': 3676, 'poste': 2853, 'contus': 3333, 'don': 1177, 'law': 2098, 'ecou': 2011, 'lo': 265, 'contente': 1165, 'jard': 3197, 'clas': 980, 'deshab': 3686, '##ppy': 330, 'differ': 887, '##ulte': 2392, '##oke': 2703, 'jeu': 2527, 'dangerous

In [3]:
vocab = tokenizer.get_vocab_size()
print('vocab:', vocab)
criterion = nn.NLLLoss(ignore_index = tokenizer.token_to_id("[PAD]"), reduction='none')
#criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.token_to_id("[PAD]"),  reduction='mean')
criterion = criterion.to(device)
model1 = Model1(vocab, vocab, criterion)

vocab: 3987


In [5]:
model1_lr=1e-3
model1_optim = torch.optim.Adam(model1.parameters(),lr=model1_lr)

In [9]:
n = len(pairs)
print(n)
train_portion=0.6
train_index = int(train_portion * n)
print(train_index)
valid_index = int(0.5 * train_index)
print(valid_index)
train_portion = pairs[0:valid_index]
un_portion = pairs[valid_index : train_index]
valid_portion = pairs[train_index:]
print(len(train_portion), len(un_portion), len(valid_portion))


train_data = get_train_dataset(train_portion[0:10], tokenizer)
un_data = get_un_dataset(un_portion[0:10], tokenizer)
valid_data = get_valid_dataset(valid_portion[0:5], tokenizer)


10599
6359
3179
3179 3180 4240
torch.Size([10, 2, 10, 1])


In [11]:
batch_size=32

In [12]:

train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), 
                        batch_size=batch_size, pin_memory=True, num_workers=0)

valid_dataloader = DataLoader(valid_data, sampler=RandomSampler(valid_data), 
                      batch_size=batch_size, pin_memory=True, num_workers=0)

un_dataloader = DataLoader(un_data, sampler=RandomSampler(un_data), 
                        batch_size=batch_size, pin_memory=True, num_workers=0)